In [ ]:
# 1. Load model and data
import shap
import pandas as pd
import joblib
import matplotlib.pyplot as plt

prefix = "ecom"  # or "bank"
model = joblib.load(f"models/{prefix}_xgb.pkl")  # assuming XGB is selected
X_test = pd.read_csv(f"data/processed/{prefix}_X_test.csv")

# 2. SHAP Explainability
explainer = shap.Explainer(model, X_test)
shap_values = explainer(X_test)

# 3. Global Summary Plot
shap.summary_plot(shap_values, X_test, show=False)
plt.savefig(f"models/{prefix}_shap_summary_plot.png")

# 4. Local Force Plot
force_plot = shap.force_plot(explainer.expected_value, shap_values[0], X_test.iloc[0], matplotlib=True)
plt.savefig(f"models/{prefix}_shap_force_plot.png")

# 5. Interpretation
# In markdown:
"""
## Insights
- Features like `Time_Since_Signup`, `Country_Mismatch`, and `Transaction_Amount` have strong contributions.
- High SHAP values indicate increased fraud likelihood.
- The force plot shows how individual feature values tilt predictions toward fraudulent classification.
"""
